In [47]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns

In [48]:
constant_patch = '11.23.1'

# 라이엇 개발자 페이지에서 최신 버전 한국 서버 챔피언 정보 가져오기
champion_info = requests.get("http://ddragon.leagueoflegends.com/cdn/" + constant_patch + "/data/ko_KR/champion.json").json()   

In [49]:
# 챔피언 정보 => DataFrame으로 변환
# champion_df는 'key'로 id값, 'name'으로 챔피언 한국어 이름을 가지는 156rows짜리 DataFrame
champion_df = pd.DataFrame(champion_info['data']).T[['key', 'name']]

# champion_df의 'key' column string => numeric 변환
champion_df['key'] = pd.to_numeric(champion_df['key'])

In [50]:
df = pd.read_csv('11.23.csv', index_col=0)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [51]:
# BIPA 데이터에 챔피언 이름 넣어주기
df = df.merge(champion_df, how = 'left', left_on = 'championId', right_on = 'key')

In [52]:
# 원하는 컬럼으로만 구성된 데이터프레임 생성
data = df[['tierRank', 'position', 'name', 'championsKilled', 'numDeaths', 'assists', 'neutralMinionsKilled',
           'turretsKilled', 'barracksKilled', 'minionsKilled', 'largestMultiKill',
           'largestCriticalStrike', 'largestKillingSpree', 'goldEarned',
           'physicalDamageDealtToChampions', 'magicDamageDealtPlayer', 'totalHeal',
           'physicalDamageTaken', 'totalDamageDealtToChampions', 'totalDamageDealt', 'totalDamageTaken',
           'neutralMinionsKilledEnemyJungle', 'neutralMinionsKilledTeamJungle', 'timeCCingOthers']]

In [53]:
# 결측치 드랍
data = data.dropna(axis=0)

In [54]:
data

,tierRank,position,name,championsKilled,numDeaths,assists,neutralMinionsKilled,turretsKilled,barracksKilled,minionsKilled,...,physicalDamageDealtToChampions,magicDamageDealtPlayer,totalHeal,physicalDamageTaken,totalDamageDealtToChampions,totalDamageDealt,totalDamageTaken,neutralMinionsKilledEnemyJungle,neutralMinionsKilledTeamJungle,timeCCingOthers
0,S31,A,징크스,9,16,21,19,3,0,204,...,31645,3492,6677,23074,39644,224711,46133,0,0,44
1,S40,S,제라스,12,3,24,13,2,0,91,...,297,61317,2470,8421,62458,167520,18224,0,0,38
2,S30,M,제드,25,6,21,21,1,1,185,...,61754,2933,7269,24763,67892,210458,39562,0,0,15
3,S425,M,야스오,5,16,8,34,2,0,280,...,17005,2018,1867,31628,21939,361021,41368,0,0,18
4,S441,S,쓰레쉬,2,16,25,0,0,0,47,...,1205,12819,5293,32367,14375,38104,53114,0,0,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5216615,S341,S,조이,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5216616,S20,A,이즈리얼,0,0,1,0,0,0,17,...,337,66,173,262,404,3626,262,0,0,0
5216617,S217,T,오른,0,0,0,0,0,0,9,...,325,49,0,306,375,5820,306,0,0,1
5216618,S476,J,렉사이,0,0,0,16,0,0,0,...,224,0,752,1107,232,7518,1552,0,0,1


In [55]:
# 티어별 데이터프레임 생성
IRON = data[data['tierRank'].str.contains('I')]
print('IRON : ', len(IRON))
BRONZE = data[data['tierRank'].str.contains('B')]
print('BRONZE : ', len(BRONZE))
SILVER = data[data['tierRank'].str.contains('S')]
print('SILVER : ', len(SILVER))
GOLD = data[data['tierRank'].str.contains('G')]
print('GOLD : ', len(GOLD))
PLATINUM = data[data['tierRank'].str.contains('P')]
print('PLATINUM : ', len(PLATINUM))
DIAMOND = data[data['tierRank'].str.contains('D')]
print('DIAMOND : ', len(DIAMOND))
m_list = ['M', 'R', 'C']
m_list = '|'.join(m_list)
ABOVE_MASTER = data[data['tierRank'].str.contains(m_list)]

IRON :  55114
BRONZE :  671861
SILVER :  1766547
GOLD :  1789950
PLATINUM :  662990
DIAMOND :  133950


In [56]:
# 포지션-챔피언별 수치 평균 구해주기
IRON_total = IRON.groupby(['position', 'name'], as_index=False).mean()
BRONZE_total = BRONZE.groupby(['position', 'name'], as_index=False).mean()
SILVER_total = SILVER.groupby(['position', 'name'], as_index=False).mean()
GOLD_total = GOLD.groupby(['position', 'name'], as_index=False).mean()
PLATINUM_total = PLATINUM.groupby(['position', 'name'], as_index=False).mean()
IRON_total = DIAMOND.groupby(['position', 'name'], as_index=False).mean()
ABOVE_MASTER_total = ABOVE_MASTER.groupby(['position', 'name'], as_index=False).mean()

In [57]:
# 본인이 원하는 위치에 데이터프레임을 csv로 저장
IRON_total.to_csv('C:\\Users\\kkandoll\\ESports_Data_Analysis\\IRON_BIPA.csv',index=False, encoding="utf-8-sig")
BRONZE_total.to_csv('C:\\Users\\kkandoll\\ESports_Data_Analysis\\BRONZE_BIPA.csv',index=False, encoding="utf-8-sig")
SILVER_total.to_csv('C:\\Users\\kkandoll\\ESports_Data_Analysis\\SILVER_BIPA.csv',index=False, encoding="utf-8-sig")
GOLD_total.to_csv('C:\\Users\\kkandoll\\ESports_Data_Analysis\\GOLD_BIPA.csv',index=False, encoding="utf-8-sig")
PLATINUM_total.to_csv('C:\\Users\\kkandoll\\ESports_Data_Analysis\\PLATINUM_BIPA.csv',index=False, encoding="utf-8-sig")
IRON_total.to_csv('C:\\Users\\kkandoll\\ESports_Data_Analysis\\DIAMOND_BIPA.csv',index=False, encoding="utf-8-sig")
ABOVE_MASTER_total.to_csv('C:\\Users\\kkandoll\\ESports_Data_Analysis\\ABOVE_MASTER_BIPA.csv',index=False, encoding="utf-8-sig")